In [ ]:
import os
import re
import shutil
import random
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
from PIL import Image
from scipy.io import wavfile
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from IPython.display import clear_output ,display

讀取excel資料，筆對時間複製對應的音檔到指定資料夾

In [ ]:
data = pd.read_excel('/home/chihli/Documents/bee_sound/feature_extraction/2023嘉義虎頭蜂黑3.xlsx', skiprows = 1, sheet_name = None, usecols="A", engine='openpyxl') 
folder1_path = '/media/chihli/TOS/sound'
folder1_files = [f for f in os.listdir(folder1_path) if f.endswith('.wav')]
output_directory = '/media/chihli/TOS/attack'

for sheet_name, df in data.items():
    for index, row in df.iterrows():
        time_data = row['出現時間']
        time = re.search( r'(.*).(.*).(.*)', str(time_data))
        time_str = time.group(0)
        time_str = time_str.replace('.', '')
        time_str = time_str.replace(':', '')
        print(time_str)
        try:
            time_str = datetime.strptime(time_str, '%H%M%S')
        except ValueError:
            print(f"Invalid time_str: {time_str}. Skipping this entry.")
            continue
        for file1 in folder1_files:
            time_detail = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*).wav', file1).group(4)
            time_detail2 = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*).wav', file1).group(5)
            date = datetime.strptime(time_detail, '%Y%m%d')
            time = datetime.strptime(time_detail2, '%H%M%S')
            if  str(date.strftime('%m%d')) == sheet_name:
                time_difference = time_str - time
                if time_difference.total_seconds() < 55 and time_difference.total_seconds() > 2 :
                    print(sheet_name, row, file1)
                    print(f'excel:{time_str}, sound:{time}')
                    print(time_difference)
                    source_file_path = os.path.join(folder1_path, file1)
                    shutil.copy2(source_file_path, output_directory)

將音檔切割成每五秒一筆

In [ ]:
# 獲取文件夾中的音頻文件列表
folder_path = '/media/chiahong/TOS2/night/attack'
audio_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('.wav'))]
# print(audio_files)

# 創建一個目錄來保存分割後的音頻文件
output_directory = '/media/chiahong/TOS2/night/attack_split'

for audio_file in audio_files:
    y, sr = librosa.load(audio_file)
    # 獲取原始文件名，去掉副檔名
    original_filename = os.path.splitext(os.path.basename(audio_file))[0]

    # 每個片段長度五秒
    segment_length = 5

    counter = 0

    # 計算每個片段採樣點數
    segment_samples = int(segment_length * sr)

    # 獲取音頻總採樣點數
    total_samples = len(y)

    # 初始化片段的起始和结束採樣點
    start_sample = 0
    end_sample = segment_samples

    while end_sample <= total_samples:
        # 提取片段
        segment = y[start_sample:end_sample]
        
        # 生成新的文件名
        output_file = os.path.join(output_directory, f'{original_filename}_{counter * segment_length}.wav')
        
        # 保存片段为WAV文件
        sf.write(output_file, segment, sr)

        # 更新起始和结束采样点，每隔5秒
        start_sample = end_sample
        end_sample = start_sample + segment_samples
        
        # 增加计数器
        counter += 1

選擇符合時間區間的音檔到指定資料夾

In [ ]:
data = pd.read_excel('/home/chihli/Documents/bee_sound/feature_extraction/2023嘉義虎頭蜂黑3.xlsx', skiprows = 1, sheet_name = None, usecols="A", engine='openpyxl') 
folder1_path = '/media/chihli/TOS/attack_split'
folder1_files = [f for f in os.listdir(folder1_path) if f.endswith('.wav')]
output_directory = '/media/chihli/TOS/attack_split_processed'

for sheet_name, df in data.items():
    for index, row in df.iterrows():
        time_data = row['出現時間']
        time = re.search( r'(.*).(.*).(.*)', str(time_data))
        time_str = time.group(0)
        time_str = time_str.replace('.', '')
        time_str = time_str.replace(':', '')
        print(time_str)
        
        try:
            time_str = datetime.strptime(time_str, '%H%M%S')
        except ValueError:
            # print(f"Invalid time_str: {time_str}. Skipping this entry.")
            continue

        for file1 in folder1_files:
                time_detail = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*)_(.*).wav', file1).group(4)
                time_detail1 = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*)_(.*).wav', file1).group(5)
                time_detail2 = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*)_(.*).wav', file1).group(6)
                date = datetime.strptime(time_detail, '%Y%m%d')
                time = datetime.strptime(time_detail1, '%H%M%S')
                sec = timedelta(seconds=int(time_detail2))
                if  str(date.strftime('%m%d')) == sheet_name:
                    time_total = time + sec
                    time_difference = time_str - time_total
                    if time_difference.total_seconds() < 5 and time_difference.total_seconds() >=0 :
                        print(f'excel:{time_str}, sound:{file1}')
                        print(time_difference)
                        source_file_path = os.path.join(folder1_path, file1)
                        shutil.copy2(source_file_path, output_directory)

隨機篩選音檔

In [ ]:
# 資料夹路徑
source_folder = '/media/chiahong/TOS/sound'
destination_folder = '/media/chiahong/TOS/am_2min_3sec/normal'
# 選擇的文件数量
N = 468

# 獲取資料夾中的檔案
all_files = [f for f in os.listdir(source_folder) if f.endswith(".wav")]

# 篩選符合時間區間的文件
selected_files = []
for file in all_files:
    if file.endswith(".wav"):
        match = re.search( r'(.*)_(.*)_(.*)_(.*)-(.*).wav', file)
        print(file)
        if match:
            time_detail = match.group(4)
            time_detail2 = match.group(5)
            date = datetime.strptime(time_detail, '%Y%m%d')
            time = datetime.strptime(time_detail2, '%H%M%S')
            date_str = date.strftime('%m%d')
            if datetime.strptime(date_str, '%m%d') > datetime.strptime('0902', '%m%d'):
                # if datetime.strptime('190000', '%H%M%S') < time or time < datetime.strptime('040000', '%H%M%S'):
                if datetime.strptime('060000', '%H%M%S') < time  < datetime.strptime('170000', '%H%M%S'):
                    selected_files.append(file)
                    print(file)
        else:
            print(f"文件名不匹配正則表達式模式：{file}")
            print(selected_files)
# 隨機選擇文件
selected_files = random.sample(selected_files, N)

# 複製指定的文件到指定的資料夾
for file_name in selected_files:
    source_file_path = os.path.join(source_folder, file_name)
    destination_file_path = os.path.join(destination_folder, file_name)
    shutil.copy2(source_file_path, destination_file_path)

print(f"成功從 {source_folder} 選擇並複製了 {N} 個文件到 {destination_folder}")